In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import json
import pandas as pd

In [3]:
from datetime import datetime, timedelta

In [75]:

# FIXME: instead of using date range, just query 3 days and concatenate results here
data1 = requests.get(
    f'https://site.api.espn.com/' +\
    f'apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20250329'
).json()
data2 = requests.get(
    f'https://site.api.espn.com/' +\
    f'apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20250330'
).json()
data3 = requests.get(
    f'https://site.api.espn.com/' +\
    f'apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20250328'
).json()
# pre-process this data to make it query-able via two-team code tuples
all_data = data1['events'] + data2['events'] + data3['events']
event_data_by_matchup_tuple = {}
for event_data in all_data:
    matchup_tuple = tuple(sorted(event_data['shortName'].split(' VS ')))
    if matchup_tuple == ('TBD', 'TBD'):
        continue
    assert len(matchup_tuple) == 2
    assert matchup_tuple not in event_data_by_matchup_tuple  # teams should only play once in the tournament
    event_data_by_matchup_tuple[matchup_tuple] = event_data


In [76]:
len(all_data), len(data1['events']), len(data2['events']), len(data3['events'])

(8, 2, 2, 4)

In [39]:
from dotenv import dotenv_values

config = dotenv_values(".env")

## ODDS API

In [5]:
from bracket import Bracket

In [17]:
bracket = Bracket.from_config(config)

[2025-04-01 15:58:36,147 - bracket - INFO] Using spread cache!
[2025-04-01 15:58:36,148 - bracket - INFO] No cache file found at: ./.spread_cache.pkl. Creating one!


In [20]:
bracket.pre_populate_events()

[2025-04-01 15:59:05,678 - bracket - INFO] Data for 64 events returned.


Found 32 determined events


In [97]:
events_by_round = bracket.to_events_by_round()

In [67]:
espn_name_to_odds_name = {}

In [68]:
def espn_team_to_odds_team(game, event):
    def teams_match(team1, team2):
        if team1 == team2:
            return True
        team1_words = team1.lower().split()
        team2_words = team2.lower().split()
        if all(
            [
                any(word1 in word2 for word2 in team2_words)
                for word1 in team1_words
            ]
        ):
            return True
        if all(
            [
                any(word2 in word1 for word1 in team1_words)
                for word2 in team2_words
            ]
        ):
            return True
        return False
    if teams_match(game['home_team'], event.home_participant.team.name):
        # ensure away matches away
        if teams_match(game['away_team'], event.away_participant.team.name):
            espn_name_to_odds_name[event.home_participant.team.name] = game['home_team']
            espn_name_to_odds_name[event.away_participant.team.name] = game['away_team']
            return {
                event.home_participant.team.name: game['home_team'],
                event.away_participant.team.name: game['away_team'],
            }
    # away team matches home team
    if teams_match(game['away_team'], event.home_participant.team.name):
        # ensure away matches home
        if teams_match(game['home_team'], event.away_participant.team.name):
            espn_name_to_odds_name[event.home_participant.team.name] = game['away_team']
            espn_name_to_odds_name[event.away_participant.team.name] = game['home_team']
            return {
                event.home_participant.team.name: game['away_team'],
                event.away_participant.team.name: game['home_team'],
            }
    return {}

In [88]:
def team_match(game, event):
    if game['home_team'] == event.home_participant.team.odds_api_name:
        # ensure away matches away
        if game['away_team'] == event.away_participant.team.odds_api_name:
            return True
    # away team matches home team
    if game['away_team'] == event.home_participant.team.odds_api_name:
        # ensure away matches home
        if game['home_team'] == event.away_participant.team.odds_api_name:
            return True
    return False

In [127]:
get_spread(s)

{"Saint Peter's Peacocks": 22.0, 'Tennessee Volunteers': -22.0}

In [117]:
import numpy as np

In [119]:
from statistics import mode


In [121]:
[1, 2, 3].index(3)

2

In [126]:
def get_spread(spreads):
    spread_point_outcomes = [spread['outcomes'][0]['point'] for spread in spreads]
    point_spread = mode(spread_point_outcomes)
    return {
        spreads[0]['outcomes'][0]['name']: point_spread,  # home
        spreads[0]['outcomes'][1]['name']: -point_spread,  # away
    }
    return spread_outcomes_away

In [111]:
s = get_spreads(event, time_min_5)

In [105]:
time_min_5

'2024-03-22T01:15:00Z'

In [103]:
s

[{'key': 'spreads',
  'last_update': '2024-03-22T01:10:09Z',
  'outcomes': [{'name': "Saint Peter's Peacocks",
    'price': -108,
    'point': 21.5},
   {'name': 'Tennessee Volunteers', 'price': -112, 'point': -21.5}]},
 {'key': 'spreads',
  'last_update': '2024-03-22T01:10:36Z',
  'outcomes': [{'name': "Saint Peter's Peacocks",
    'price': -109,
    'point': 22.0},
   {'name': 'Tennessee Volunteers', 'price': -112, 'point': -22.0}]},
 {'key': 'spreads',
  'last_update': '2024-03-22T01:10:09Z',
  'outcomes': [{'name': "Saint Peter's Peacocks",
    'price': -109,
    'point': 22.0},
   {'name': 'Tennessee Volunteers', 'price': -112, 'point': -22.0}]},
 {'key': 'spreads',
  'last_update': '2024-03-22T01:09:35Z',
  'outcomes': [{'name': "Saint Peter's Peacocks",
    'price': -115,
    'point': 22.5},
   {'name': 'Tennessee Volunteers', 'price': -105, 'point': -22.5}]},
 {'key': 'spreads',
  'last_update': '2024-03-22T01:09:56Z',
  'outcomes': [{'name': "Saint Peter's Peacocks",
    'pric

In [101]:
def get_spreads(event, date_str):
    spreads_past = requests.get(
        f'https://api.the-odds-api.com/v4/historical/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=american&date={date_str}'
    ).json()
    spreads = []
    for game in spreads_past['data']:
        # espn_to_odds_names = espn_team_to_odds_team(game, event)
        if team_match(game, event):
            for bookmaker in game['bookmakers']:
                spreads.append(bookmaker['markets'][0])#['outcomes'][0]['point'])
            break
    if len(spreads) == 0:
        print('no match')
        teams = [(game['home_team'], game['away_team']) for game in spreads_past['data']]
        print(teams)
        pass
    return spreads

In [99]:
for round in (1,):
    for event in events_by_round[round]:
        time_min_5 = (event.estimated_start_time - timedelta(minutes=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
        print(event.home_participant.team.name, event.away_participant.team.name, time_min_5)
        print('\t', get_past_odds(event, time_min_5))

UConn Huskies Stetson Hatters 2024-03-22T19:17:00Z
	 [27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5]
Florida Atlantic Owls Northwestern Wildcats 2024-03-22T16:10:00Z
	 [-4.5, -3.5, -4.5, -4.5, -4.5, -4.5, -4.0, -4.0, -4.0, -4.5, -4.5, -4.0, -4.0, -4.5]
San Diego State Aztecs UAB Blazers 2024-03-22T17:40:00Z
	 [-6.0, -6.5, -6.5, -6.5, -6.5, -6.5, -6.0, -6.5, -6.5, -6.0, -6.5, -6.0, -6.5, -6.5]
Auburn Tigers Yale Bulldogs 2024-03-22T20:28:00Z
	 [-14.0, -14.0, -13.5, -13.5, -14.0, -14.0, -14.5, -13.5, -13.5, -14.0, -14.0, -14.0, -13.5, -13.5]
BYU Cougars Duquesne Dukes 2024-03-21T16:35:00Z
	 [-9.5, -9.5, -9.5, -9.0, -9.5, -9.5, -9.0, -9.5, -9.5, -9.0, -9.5, -9.5, -9.5]
Illinois Fighting Illini Morehead State Eagles 2024-03-21T19:25:00Z
	 [-10.5, -11.0, -11.5, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0]
Washington State Cougars Drake Bulldogs 2024-03-22T02:07:00Z
	 [-1.0, -1.5, -1.0, -1.0, -1.5, -1.5, -1.0, -1.0, -1.0, -1.0, -1

In [90]:
len(espn_name_to_odds_name)

64

In [91]:
for event in events_by_round[1]:
    if event.home_participant.team.name not in espn_name_to_odds_name:
        print(event.home_participant.team.name)
    if event.away_participant.team.name not in espn_name_to_odds_name:
        print(event.away_participant.team.name)

In [72]:
espn_name_to_odds_name['Arizona Wildcats'] = 'Arizona Wildcats'
espn_name_to_odds_name['Long Beach State Beach'] = 'Long Beach St 49ers'

In [73]:
len(espn_name_to_odds_name)

64

In [74]:
bracket_df = pd.read_csv('mm-with-team-codes.csv')
bracket_df.head()

,participant_name,team_name,seed,team_name_norm,team_code
0,Scott,UConn Huskies,1,UConn,CONN
1,Katz,Stetson Hatters,16,Stetson,STET
2,Molloy,Florida Atlantic Owls,8,Florida Atlantic,FAU
3,John + Tyler,Northwestern Wildcats,9,Northwestern,NU
4,Eddie,San Diego State Aztecs,5,San Diego St,SDSU


In [76]:
bracket_df['odds_api_team_name'] = bracket_df.team_name.apply(lambda n: espn_name_to_odds_name[n])
bracket_df.head()

,participant_name,team_name,seed,team_name_norm,team_code,odds_api_team_name
0,Scott,UConn Huskies,1,UConn,CONN,UConn Huskies
1,Katz,Stetson Hatters,16,Stetson,STET,Stetson Hatters
2,Molloy,Florida Atlantic Owls,8,Florida Atlantic,FAU,Florida Atlantic Owls
3,John + Tyler,Northwestern Wildcats,9,Northwestern,NU,Northwestern Wildcats
4,Eddie,San Diego State Aztecs,5,San Diego St,SDSU,San Diego St Aztecs


In [77]:
bracket_df.isna().sum()

participant_name      0
team_name             0
seed                  0
team_name_norm        0
team_code             0
odds_api_team_name    0
dtype: int64

In [79]:
bracket_df_clean = bracket_df.drop(columns=['team_name_norm'])
bracket_df_clean.head()

,participant_name,team_name,seed,team_code,odds_api_team_name
0,Scott,UConn Huskies,1,CONN,UConn Huskies
1,Katz,Stetson Hatters,16,STET,Stetson Hatters
2,Molloy,Florida Atlantic Owls,8,FAU,Florida Atlantic Owls
3,John + Tyler,Northwestern Wildcats,9,NU,Northwestern Wildcats
4,Eddie,San Diego State Aztecs,5,SDSU,San Diego St Aztecs


In [81]:
bracket_df_clean.to_csv('mm-with-team-codes.csv', index=False)

In [17]:
missing_event = events_by_round[2][1]
time_min_5 = (missing_event.estimated_start_time - timedelta(minutes=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
get_past_odds(missing_event.home_participant.team.name, time_min_5)

[]

In [12]:
print(event.estimated_start_time.strftime('%Y-%m-%dT%H:%M:%SZ'))
print((event.estimated_start_time - timedelta(minutes=5)).strftime('%Y-%m-%dT%H:%M:%SZ'))

NameError: name 'event' is not defined

In [67]:
len(bracket._events_to_process)

15

In [66]:
bracket.participants[0].team.name, bracket.participants[0].team.name, 

'UConn Huskies'

In [21]:
# {
#     "key": "basketball_ncaab",
#     "group": "Basketball",
#     "title": "NCAAB",
#     "description": "US College Basketball",
#     "active": true,
#     "has_outrights": false
#   },

In [29]:
from_str = datetime(2025, 3, 9, 4, 15).strftime('%Y-%m-%dT%H:%M:%SZ')  # 2025-03-09T00%3A00%3A00Z
to_str = datetime(2025, 4, 20, 4, 15).strftime('%Y-%m-%dT%H:%M:%SZ')  # 2025-04-20T00%3A00%3A00Z

In [30]:
spreads_upcoming = requests.get(
    f'https://api.the-odds-api.com/v4/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=decimal&commenceTimeFrom={from_str}&commenceTimeTo={to_str}'
).json()


In [31]:
len(spreads_upcoming)

13

In [25]:
spreads_upcoming[0].keys()

dict_keys(['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team', 'bookmakers'])

In [33]:
import numpy as np

In [34]:
odds_long_team_names = []
for game in spreads_upcoming:
    odds_long_team_names.append(game["home_team"])
    odds_long_team_names.append(game["away_team"])
    print(f'{game["home_team"]} vs. {game["away_team"]}')
    print(f'\ttime start: {game["commence_time"]}')
    spreads = []
    for bookmaker in game['bookmakers']:
        spreads.append(bookmaker['markets'][0]['outcomes'][0]['point'])
    print(f'\t{spreads}')
    print(f'\tspread mean: {np.mean(spreads)}')
    print(f'\tspread std: {np.std(spreads)}')

UCF Knights vs. Oregon St Beavers
	time start: 2025-04-01T21:38:06Z
	[6.0, 0.5, 1.5]
	spread mean: 2.6666666666666665
	spread std: 2.3921166824012206
UC Irvine Anteaters vs. North Texas Mean Green
	time start: 2025-04-01T23:00:00Z
	[-1.5, -1.5, -1.5, -1.0, -1.5, -1.5]
	spread mean: -1.4166666666666667
	spread std: 0.18633899812498245
Villanova Wildcats vs. Colorado Buffaloes
	time start: 2025-04-02T00:30:00Z
	[4.5, 4.5, 5.0, 4.5, 4.5]
	spread mean: 4.6
	spread std: 0.19999999999999998
Loyola (Chi) Ramblers vs. Chattanooga Mocs
	time start: 2025-04-02T01:30:00Z
	[2.5, 2.5, 2.5, 2.5, 2.5, 2.5]
	spread mean: 2.5
	spread std: 0.0
USC Trojans vs. Tulane Green Wave
	time start: 2025-04-02T03:00:00Z
	[8.0, 8.0, 8.0, 7.5, 8.5]
	spread mean: 8.0
	spread std: 0.31622776601683794
Boise State Broncos vs. Butler Bulldogs
	time start: 2025-04-02T23:00:00Z
	[-6.5, -6.5, -6.5, -6.5, -6.5]
	spread mean: -6.5
	spread std: 0.0
Georgetown Hoyas vs. Nebraska Cornhuskers
	time start: 2025-04-03T01:30:00Z
	[

In [60]:
date_str = datetime(2025, 3, 18, 15, 15).strftime('%Y-%m-%dT%H:%M:%SZ') # START DATE OF MARCH MADNESS
spreads_past = requests.get(
    f'https://api.the-odds-api.com/v4/historical/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=american&date={date_str}'
).json()

In [61]:
len(spreads_past['data'])

61

In [62]:
spreads_past['data'][0].keys()

dict_keys(['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team', 'bookmakers'])

In [64]:
odds_long_team_names = []
for game in spreads_past['data']:
    odds_long_team_names.append(game["home_team"])
    odds_long_team_names.append(game["away_team"])
    # print(f'{game["home_team"]} vs. {game["away_team"]}')
    # print(f'\ttime start: {game["commence_time"]}')
    spreads = []
    for bookmaker in game['bookmakers']:
        spreads.append(bookmaker['markets'][0]['outcomes'][0]['point'])
    # print(f'\t{spreads}')
    # print(f'\tspread mean: {np.mean(spreads)}')
    # print(f'\tspread std: {np.std(spreads)}')
len(odds_long_team_names), len(set(odds_long_team_names))

(122, 109)

## ESPN API

In [24]:
data_from_march = requests.get(
    'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=202503'
).json()

In [25]:
len(data_from_march['events'])

64

In [54]:
df_teams = pd.read_csv('mm-with-team-codes.csv')

In [26]:
short_teams, long_teams = [], []
for event in data_from_march['events']:
    s_teams = event['shortName'].split(' VS ')
    assert len(s_teams) == 2
    l_teams = event['name'].split(' at ')
    assert len(l_teams) == 2
    short_teams += s_teams
    long_teams += l_teams

long_to_short = dict(zip(long_teams, short_teams))


In [27]:
long_to_short

{'North Carolina Tar Heels': 'UNC',
 'San Diego State Aztecs': 'SDSU',
 'St. Francis (PA) Red Flash': 'SFPA',
 'Alabama State Hornets': 'ALST',
 'Xavier Musketeers': 'XAV',
 'Texas Longhorns': 'TEX',
 "Mount St. Mary's Mountaineers": 'MSM',
 'American University Eagles': 'AMER',
 'SIU Edwardsville Cougars': 'SIUE',
 'Houston Cougars': 'HOU',
 'Auburn Tigers': 'AUB',
 'Wofford Terriers': 'WOF',
 'Tennessee Volunteers': 'TENN',
 'Omaha Mavericks': 'OMA',
 "St. John's Red Storm": 'SJU',
 'Montana Grizzlies': 'MONT',
 'Wisconsin Badgers': 'WIS',
 'UNC Wilmington Seahawks': 'UNCW',
 'Texas Tech Red Raiders': 'TTU',
 'High Point Panthers': 'HPU',
 'Purdue Boilermakers': 'PUR',
 'Yale Bulldogs': 'YALE',
 'Texas A&M Aggies': 'TA&M',
 'McNeese Cowboys': 'MCN',
 'Clemson Tigers': 'CLEM',
 'UC San Diego Tritons': 'UCSD',
 'Michigan Wolverines': 'MICH',
 'VCU Rams': 'VCU',
 'BYU Cougars': 'BYU',
 'Drake Bulldogs': 'DRKE',
 'Missouri Tigers': 'MIZ',
 'Arkansas Razorbacks': 'ARK',
 'Kansas Jayhawks'

In [28]:
len(long_to_short)

68

In [71]:
set(long_to_short.keys())

{'Akron Zips',
 'Alabama Crimson Tide',
 'Alabama State Hornets',
 'American University Eagles',
 'Arizona Wildcats',
 'Arkansas Razorbacks',
 'Auburn Tigers',
 'BYU Cougars',
 'Baylor Bears',
 'Bryant Bulldogs',
 'Clemson Tigers',
 'Colorado State Rams',
 'Creighton Bluejays',
 'Drake Bulldogs',
 'Duke Blue Devils',
 'Florida Gators',
 'Georgia Bulldogs',
 'Gonzaga Bulldogs',
 'Grand Canyon Lopes',
 'High Point Panthers',
 'Houston Cougars',
 'Illinois Fighting Illini',
 'Iowa State Cyclones',
 'Kansas Jayhawks',
 'Kentucky Wildcats',
 'Liberty Flames',
 'Lipscomb Bisons',
 'Louisville Cardinals',
 'Marquette Golden Eagles',
 'Maryland Terrapins',
 'McNeese Cowboys',
 'Memphis Tigers',
 'Michigan State Spartans',
 'Michigan Wolverines',
 'Mississippi State Bulldogs',
 'Missouri Tigers',
 'Montana Grizzlies',
 "Mount St. Mary's Mountaineers",
 'New Mexico Lobos',
 'Norfolk State Spartans',
 'North Carolina Tar Heels',
 'Oklahoma Sooners',
 'Ole Miss Rebels',
 'Omaha Mavericks',
 'Orego

In [53]:
sorted(long_to_short.keys())

['Akron Zips',
 'Alabama Crimson Tide',
 'Arizona Wildcats',
 'Auburn Tigers',
 'BYU Cougars',
 'Baylor Bears',
 'Boise State Broncos',
 'Charleston Cougars',
 'Clemson Tigers',
 'Colgate Raiders',
 'Colorado Buffaloes',
 'Colorado State Rams',
 'Creighton Bluejays',
 'Dayton Flyers',
 'Drake Bulldogs',
 'Duke Blue Devils',
 'Duquesne Dukes',
 'Florida Atlantic Owls',
 'Florida Gators',
 'Gonzaga Bulldogs',
 'Grambling Tigers',
 'Grand Canyon Lopes',
 'Houston Cougars',
 'Howard Bison',
 'Illinois Fighting Illini',
 'Iowa State Cyclones',
 'James Madison Dukes',
 'Kansas Jayhawks',
 'Kentucky Wildcats',
 'Long Beach State Beach',
 'Longwood Lancers',
 'Marquette Golden Eagles',
 'McNeese Cowboys',
 'Michigan State Spartans',
 'Mississippi State Bulldogs',
 'Montana State Bobcats',
 'Morehead State Eagles',
 'NC State Wolfpack',
 'Nebraska Cornhuskers',
 'Nevada Wolf Pack',
 'New Mexico Lobos',
 'North Carolina Tar Heels',
 'Northwestern Wildcats',
 'Oakland Golden Grizzlies',
 'Oregon 

In [69]:
set(odds_long_team_names)

{'Akron Zips',
 'Alabama Crimson Tide',
 'Alabama St Hornets',
 'American Eagles',
 'Arizona Wildcats',
 'Arkansas Razorbacks',
 'Arkansas St Red Wolves',
 'Army Knights',
 'Auburn Tigers',
 'BYU Cougars',
 'Baylor Bears',
 'Bradley Braves',
 'Bryant Bulldogs',
 'CSU Northridge Matadors',
 'Chattanooga Mocs',
 'Clemson Tigers',
 'Colorado St Rams',
 'Creighton Bluejays',
 'Dayton Flyers',
 'Drake Bulldogs',
 'Duke Blue Devils',
 'Elon Phoenix',
 'Florida Atlantic Owls',
 'Florida Gators',
 'Furman Paladins',
 'George Mason Patriots',
 'Georgia Bulldogs',
 'Georgia Tech Yellow Jackets',
 'Gonzaga Bulldogs',
 'Grand Canyon Antelopes',
 'High Point Panthers',
 'Houston Cougars',
 'Illinois Fighting Illini',
 'Illinois St Redbirds',
 'Incarnate Word Cardinals',
 'Iowa State Cyclones',
 'Jacksonville Dolphins',
 'Jacksonville St Gamecocks',
 'Kansas Jayhawks',
 'Kent State Golden Flashes',
 'Kentucky Wildcats',
 'Liberty Flames',
 'Lipscomb Bisons',
 'Louisville Cardinals',
 'Loyola (Chi) R

In [66]:
for name in set(odds_long_team_names):
    print(name, name in long_to_short)

Dayton Flyers False
Furman Paladins False
UC San Diego Tritons True
Stanford Cardinal False
Maryland Terrapins True
Illinois Fighting Illini True
Utah State Aggies True
Troy Trojans True
Ole Miss Rebels True
Army Knights False
Akron Zips True
North Texas Mean Green False
Manhattan Jaspers False
Saint Mary's Gaels True
Wisconsin Badgers True
Saint Joseph's Hawks False
Santa Clara Broncos False
Colorado St Rams False
UC Irvine Anteaters False
St. Francis (PA) Red Flash True
Northern Arizona Lumberjacks False
Georgia Bulldogs True
Arkansas Razorbacks True
Kentucky Wildcats True
Alabama St Hornets False
New Mexico Lobos True
Robert Morris Colonials True
N Colorado Bears False
Saint Louis Billikens False
Jacksonville St Gamecocks False
St. Bonaventure Bonnies False
VCU Rams True
Mississippi St Bulldogs False
Drake Bulldogs True
San José St Spartans False
Wofford Terriers True
Bryant Bulldogs True
San Francisco Dons False
UC Riverside Highlanders False
Chattanooga Mocs False
Wichita St Shock

In [45]:
odds_long_team_names[0] in long_to_short

True

In [58]:
short_to_long = dict(zip(short_teams, long_teams))

In [60]:
df_teams['team_name'] = df_teams['team_code'].apply(lambda c: short_to_long[c])

In [61]:
df_teams.to_csv('mm-with-team-codes.csv', index=False)

In [42]:
short_to_long

{'STET': 'Stetson Hatters',
 'CONN': 'UConn Huskies',
 'GRAM': 'Grambling Tigers',
 'PUR': 'Purdue Boilermakers',
 'LONG': 'Longwood Lancers',
 'HOU': 'Houston Cougars',
 'WKU': 'Western Kentucky Hilltoppers',
 'MARQ': 'Marquette Golden Eagles',
 'COLG': 'Colgate Raiders',
 'BAY': 'Baylor Bears',
 'YALE': 'Yale Bulldogs',
 'AUB': 'Auburn Tigers',
 'UVM': 'Vermont Catamounts',
 'DUKE': 'Duke Blue Devils',
 'COFC': 'Charleston Cougars',
 'ALA': 'Alabama Crimson Tide',
 'UAB': 'UAB Blazers',
 'SDSU': 'San Diego State Aztecs',
 'JMU': 'James Madison Dukes',
 'WIS': 'Wisconsin Badgers',
 'GCU': 'Grand Canyon Lopes',
 'SMC': "Saint Mary's Gaels",
 'UNM': 'New Mexico Lobos',
 'CLEM': 'Clemson Tigers',
 'COLO': 'Colorado Buffaloes',
 'FLA': 'Florida Gators',
 'NU': 'Northwestern Wildcats',
 'FAU': 'Florida Atlantic Owls',
 'TA&M': 'Texas A&M Aggies',
 'NEB': 'Nebraska Cornhuskers',
 'TCU': 'TCU Horned Frogs',
 'USU': 'Utah State Aggies',
 'MSU': 'Michigan State Spartans',
 'UNC': 'North Caroli

In [ ]:
event_types = set()
for event in data_from_march['events']:
    event_types.add(event['season']['slug'])

In [ ]:
test_df = pd.read_csv('monkey-madness-teams.csv')

In [ ]:
def normalize_team_name(team_name):
    n = team_name.strip(".").strip()
    if n.endswith("'s"):
        return n[:-2]
    return n

In [ ]:
def team_name_to_short_code(team_name_norm):
    short_team_code = None
    for long_team, short_team in long_to_short.items():
        if team_name_norm.lower() in long_team.lower() or long_team.lower() in team_name_norm.lower():
            short_team_code = short_team
        elif short_team.lower() in team_name_norm.lower():
            short_team_code = short_team
    return short_team_code

In [ ]:
test_df['team_name_norm'] = test_df.team_name.apply(normalize_team_name)
test_df['team_code'] = test_df.team_name_norm.apply(team_name_to_short_code)

In [ ]:
# fix empty and duplicates

In [ ]:
code_counts = dict(test_df.team_code.value_counts())

In [ ]:
dup_codes = [key for key, value in code_counts.items() if value > 1]

In [ ]:
dup_codes

In [ ]:
no_code = test_df[pd.isna(test_df.team_code)]
print(len(no_code))
no_code.head(15)

In [ ]:
dup_code = test_df[test_df.team_code.apply(lambda c: c in dup_codes)]
print(len(dup_code))
dup_code.head(15)

In [ ]:
fixes = {
    'Molloy (2)': 'FLA',
    'Nicolai': 'MORE',
    'F-Cup': 'LBSU',
    'Scott (2)': 'DUKE',
    'Cimi (2)': 'UK',
    'Peralo (2)': 'CSU',
    'Leff (2)': 'TEX'
}

In [ ]:
test_df['team_code'] = test_df.apply(
    lambda row: fixes[row.participant_name] if row.participant_name in fixes else row.team_code,
    axis=1
)

In [ ]:
test_df.isna().sum()

In [ ]:
len(test_df.team_code.unique())

In [ ]:
test_df.team_code.value_counts()

In [ ]:
test_df[test_df.team_code.values == 'TEX'].head()

In [ ]:
test_df[test_df.team_code.values == 'TTU'].head()

In [ ]:
len(test_df.team_code.unique())

In [ ]:
test_df.to_csv('mm-with-team-codes.csv', index=False)

In [ ]:
# manually changed to TTU^

In [ ]:
df_clean = pd.read_csv('mm-with-team-codes.csv')
assert len(df_clean.team_code.unique()) == 64

In [ ]:
from construct_bracket import Bracket

In [ ]:
bracket = Bracket.from_csv('mm-with-team-codes.csv')

In [ ]:
bracket

In [ ]:
data_from_march.keys()

In [ ]:
is_finished = data_from_march['events'][0]['status']['type']['completed']  # boolean
status = data_from_march['events'][0]['status']['type']['name']
# change last 0 to 1 for second team
team_to_score = {}
for i in range(2):
    score = data_from_march['events'][0]['competitions'][0]['competitors'][i]['score']
    team_code = data_from_march['events'][0]['competitions'][0]['competitors'][i]['team']['abbreviation']
    team_to_score[team_code] = score
status, is_finished, team_to_score

In [ ]:
data_from_march['events'][0]['competitions'][0]['competitors'][0]['winner']